<a href="https://colab.research.google.com/github/baalasangar/MLAI/blob/master/08_RecommendationSystem/Assignment/recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#### 1) Importing Liberaries & reading all data





In [185]:
import pandas as pd 
pd.set_option('display.max_columns', None)

import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
#%matplotlib inline
sns.set(style="whitegrid")

In [186]:
data_rev1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Greatelearning/RecommendationSys/phone_user_review_file_1.csv",encoding='latin1')
data_rev2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Greatelearning/RecommendationSys/phone_user_review_file_2.csv",encoding='latin1')
data_rev3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Greatelearning/RecommendationSys/phone_user_review_file_3.csv",encoding='latin1')
data_rev4 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Greatelearning/RecommendationSys/phone_user_review_file_4.csv",encoding='latin1')
data_rev5 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Greatelearning/RecommendationSys/phone_user_review_file_5.csv",encoding='latin1')
data_rev6 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Greatelearning/RecommendationSys/phone_user_review_file_6.csv",encoding='latin1')



In [187]:
# Check the Details for the data
data_list = [data_rev1,data_rev2,data_rev3,data_rev4,data_rev5,data_rev6]
for data_object in data_list:
  print(data_object.columns)
  print(data_object.head())

Index(['phone_url', 'date', 'lang', 'country', 'source', 'domain', 'score',
       'score_max', 'extract', 'author', 'product'],
      dtype='object')
                        phone_url       date lang country            source  \
0  /cellphones/samsung-galaxy-s8/   5/2/2017   en      us  Verizon Wireless   
1  /cellphones/samsung-galaxy-s8/  4/28/2017   en      us       Phone Arena   
2  /cellphones/samsung-galaxy-s8/   5/4/2017   en      us            Amazon   
3  /cellphones/samsung-galaxy-s8/   5/2/2017   en      us           Samsung   
4  /cellphones/samsung-galaxy-s8/  5/11/2017   en      us  Verizon Wireless   

                domain  score  score_max  \
0  verizonwireless.com   10.0       10.0   
1       phonearena.com   10.0       10.0   
2           amazon.com    6.0       10.0   
3          samsung.com    9.2       10.0   
4  verizonwireless.com    4.0       10.0   

                                             extract       author  \
0  As a diehard Samsung fan who has had 

##### Merge the provided CSVs into one data-frame.


In [188]:
data = pd.concat(data_list)
data.reset_index(inplace=True)
data.drop(columns=["index"],inplace=True)
data.tail(n=10000)


,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
1405133,/cellphones/sony-cmd-z5/,12/3/2001,en,gb,Dooyoo,dooyoo.co.uk,6.0,10.0,This a realtivly new phone and has an importan...,anonym,Sony CMD-Z5
1405134,/cellphones/sony-cmd-z5/,10/22/2001,en,gb,Dooyoo,dooyoo.co.uk,10.0,10.0,"THE best phone on the market, classy and featu...",timbos,Sony CMD-Z5
1405135,/cellphones/sony-cmd-z5/,9/25/2001,en,gb,Dooyoo,dooyoo.co.uk,10.0,10.0,When looking at the Sony CMD-Z5 it can be very...,M_J_Carrier,Sony CMD-Z5
1405136,/cellphones/sony-cmd-z5/,8/12/2001,en,gb,Dooyoo,dooyoo.co.uk,6.0,10.0,I`ve had the Sony CMD-Z5 for just about a year...,Verbal,Sony CMD-Z5
1405137,/cellphones/sony-cmd-z5/,7/25/2001,en,gb,Dooyoo,dooyoo.co.uk,8.0,10.0,Now that my current phone's contract is immine...,curly00,Sony CMD-Z5
...,...,...,...,...,...,...,...,...,...,...,...
1415128,/cellphones/alcatel-ot-club_1187/,5/12/2000,de,de,Ciao,ciao.de,2.0,10.0,Weil mein Onkel bei ALcatel arbeitet habe ich ...,david.paul,Alcatel Club Plus Handy
1415129,/cellphones/alcatel-ot-club_1187/,5/11/2000,de,de,Ciao,ciao.de,10.0,10.0,Hy Liebe Leserinnen und Leser!! Ich habe seit ...,Christiane14,Alcatel Club Plus Handy
1415130,/cellphones/alcatel-ot-club_1187/,5/4/2000,de,de,Ciao,ciao.de,2.0,10.0,"Jetzt hat wohl Alcatell gedacht ,sie machen wa...",michaelawr,Alcatel Club Plus Handy
1415131,/cellphones/alcatel-ot-club_1187/,5/1/2000,de,de,Ciao,ciao.de,8.0,10.0,Ich bin seit 2 Jahren (stolzer) Besitzer eines...,claudia0815,Alcatel Club Plus Handy


#### Exploratory Data Analysis

In [189]:
# Create a copy of the data for analysis
data_analysis = data.copy()

##### Check a few observations and shape of the data-frame.


In [190]:
print("Shape of the data - ",data_analysis.shape)
print("Columns of data - ",data_analysis.columns)

Shape of the data -  (1415133, 11)
Columns of data -  Index(['phone_url', 'date', 'lang', 'country', 'source', 'domain', 'score',
       'score_max', 'extract', 'author', 'product'],
      dtype='object')


In [191]:
# Random check on data for 5o records
data_analysis.sample(n=25,random_state=612)

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
1381066,/cellphones/samsung-mm-a700-sph-a700/,9/18/2004,en,us,Phone Scoop,phonescoop.com,3.0,10.0,Sprint missed the boat on the fundamentals wit...,GrandWazoo,MM-A700 / SPH-A700
159224,/cellphones/lg-g5/,1/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,"Just got this a couple of weeks ago, really ni...",Hunts1,LG G5
166941,/cellphones/microsoft-lumia-640-dual-sim/,3/10/2016,ru,ru,Yandex,market.yandex.ru,6.0,10.0,Ð¡ÑÐµÐ´Ð½Ð¸Ð¹ ÑÐ¼Ð°ÑÑÑÐ¾Ð½. ÐÐ· Ð¿Ð»ÑÑ...,NaN,Microsoft Lumia 640 3G Dual Sim
340422,/cellphones/samsung-galaxy-s5/,5/24/2016,en,in,Flipkart,flipkart.com,8.0,10.0,Good...reasonable. .original product of Samsun...,Sandeep Kumar...,"SAMSUNG Galaxy S5 (Copper Gold, 16 GB)"
290110,/cellphones/samsung-galaxy-note-4/,11/13/2014,en,ca,Amazon,amazon.ca,10.0,10.0,Shipped very fast and came with everything you...,mike hawdon,"Tech Armor Galaxy Note 4 Screen Protector, Tec..."
1166492,/cellphones/nokia-e71/,2/20/2009,en,us,Amazon,amazon.com,4.0,10.0,"Phone arrived, and the initial looks of it wer...",The mother of all prduct reviews,Nokia E71x Unlocked Phone with QWERTY Keyboard...
264600,/cellphones/apple-iphone-6/,11/13/2014,en,us,Amazon,amazon.com,8.0,10.0,The phone was to my expectations and I was hap...,Amazon Customer,Apple iPhone 6 16GB Factory Unlocked GSM 4G LT...
1275859,/cellphones/nokia-6680/,1/21/2007,en,gb,Amazon,amazon.co.uk,8.0,10.0,Its hot chrome packing slick phone albeit it b...,Amazon Customer,Nokia 6680 Smartphone
416067,/cellphones/sony-xperia-z3-compact/,11/25/2014,en,gb,Amazon,amazon.co.uk,6.0,10.0,I'd like to say this phone is great- and so mu...,Jsphstry,Sony Xperia Z3 Compact UK SIM-Free Smartphone ...
306382,/cellphones/acer-liquid-z630/,12/21/2015,fr,fr,Amazon,amazon.fr,2.0,10.0,laissant un samsung s5 j'ai pris celui la car ...,Client d'Amazon,Acer Liquid Z630 Smartphone dÃ©bloquÃ© 4G (Ecr...


##### formating the phone url Columns

In [192]:
# formating the phone url Columns

# Check is all the rows starts with "cellphones"
print(data_analysis['phone_url'].str.startswith('/cellphones/').value_counts() )
data_analysis['phone_url'] = data_analysis['phone_url'].str.replace('/cellphones/','')
data_analysis['phone_url'] = data_analysis['phone_url'].str[:-1]
data_analysis['phone_url'].value_counts()

True    1415133
Name: phone_url, dtype: int64


samsung-galaxy-s-iii         17093
apple-iphone-5s              16379
samsung-galaxy-s6            16145
samsung-galaxy-s5            16082
samsung-galaxy-s7-edge       15917
                             ...  
samsung-corby-tv-sch-f339        1
samsung-chrono-2                 1
siemens-cl55                     1
verykool-s400                    1
blackberry-6280                  1
Name: phone_url, Length: 5556, dtype: int64

##### Analysis the Column "author" 

In [193]:
#pd.set_option('display.max_rows', None)
print("Not of observation with empty - ",data_analysis["author"].isna().sum())
print("Not of observation with empty % - ",(data_analysis["author"].isna().sum() / data_analysis.shape[0])*100) 

Not of observation with empty -  63202
Not of observation with empty % -  4.466152651376231


Since only 1000000 are request for analysis, removing the observation where author names are empty





 

In [194]:

data_analysis.dropna(subset=["author"],inplace=True)
data_analysis.isna().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        60893
score_max    60893
extract      15515
author           0
product          0
dtype: int64

##### Analysis the Column "Score and score_max"

In [195]:
##### Analysis the Column "Score and score_max"

In [196]:
print("Not of observation with empty - ",data_analysis["score"].isna().sum())
print("Not of observation with empty % - ",(data_analysis["score"].isna().sum() / data_analysis.shape[0])*100) 

Not of observation with empty -  60893
Not of observation with empty % -  4.504149989903331


Since only 1000000 are request for analysis, removing the observation where score names are empty

In [197]:
data_analysis.dropna(subset=["score"],inplace=True)
data_analysis.isna().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score            0
score_max        0
extract      15121
author           0
product          0
dtype: int64

In [198]:
# is there any observation with score greater than max score ?
data_analysis.query("score > score_max").count().sum()


0

 - is there any observation with score greater than max score ? -NO

###### Round off scores to the nearest integers



In [199]:
data_analysis["score_round"] = np.rint(data_analysis["score"])
data_analysis["score_round"].unique()

array([10.,  6.,  9.,  4.,  8.,  2.,  7.,  5.,  3.,  1.,  0.])

In [200]:
# is the scoring is done on same scale  ( 1 -10) ?  
data_analysis["score_max"].value_counts()

10.0    1291038
Name: score_max, dtype: int64

- is the scoring is done on same scale  ( 1 -10) ?  - YES

##### Dropping irrelevant features & removing duplicate columns

In [201]:
# Removing the duplicate rows - when phone_url,author & score are same
rows_with_duplicate = int(data_analysis.shape[0])

print("# of Observations - ",rows_with_duplicate)
data_analysis.drop_duplicates(subset=['phone_url','author','score'],keep='first',inplace=True)

rows_without_duplicate = int(data_analysis.shape[0])
print("# of Observations after removing duplicate - ",rows_without_duplicate)

print("% of rows duplicated rows removed - " , ((rows_with_duplicate - rows_without_duplicate)/rows_with_duplicate)*100)

# of Observations -  1291038
# of Observations after removing duplicate -  1114575
% of rows duplicated rows removed -  13.668304108786883


- 13% duplicated rows have been removed , it's OK since only 10L rows are to be considered for the analysis. 

In [202]:
# 'phone_url','author','score' are considered for the analysis on other rows are removed.
data_analysis = data_analysis[['phone_url','author','score']]
data_analysis.head()

,phone_url,author,score
0,samsung-galaxy-s8,CarolAnn35,10.0
1,samsung-galaxy-s8,james0923,10.0
2,samsung-galaxy-s8,R. Craig,6.0
3,samsung-galaxy-s8,Buster2020,9.2
4,samsung-galaxy-s8,S Ate Mine,4.0


##### Sampling 1000000 rows

In [203]:
data_analysis = data_analysis.sample(n=1000000,random_state=612)
print("Check the shape ", data_analysis.shape)
print(" missing values ", data_analysis.isna().sum().sum())

Check the shape  (1000000, 3)
 missing values  0


### 2) Answer the following questions 

1.   Identify the most rated features
2.   Identify the users with most number of reviews.
3.   Select the data with products having more than 50 ratings and users who have given more than 50 ratings. Report the shape of the final
dataset

##### Selecting products with 50 rating and 

In [205]:
data_groupby_phone = data_analysis.groupby(by="phone_url").count()
data_groupby_phone = data_groupby_phone.query("author > 50 & score > 50 ").sort_values(by=['author','score'],ascending=False)
data_groupby_phone.head()

,author,score
phone_url,,
samsung-galaxy-s5,11758,11758
samsung-galaxy-s6,11421,11421
motorola-moto-g,10755,10755
samsung-galaxy-s7-edge,10714,10714
apple-iphone-5s,10537,10537


In [212]:
data_analysis = data_analysis[data_analysis["phone_url"].isin(data_groupby_phone.index)]
data_analysis.shape

(950014, 3)

- 950014 observation selected further anaysis